<a href="https://colab.research.google.com/github/MWFK/NLP-Semantic-Similarity/blob/main/NLP-Semantic-Similarity/ClinicalTrials/Data%20Engineering/EligibilityCriteria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [75]:
import re
import pandas as pd
# pd.set_option('display.max_columns', None)  
# pd.set_option('display.max_colwidth', None)
import requests
from itertools import compress
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from google.colab import files

In [59]:
####### Search Expression #######
# None

####### Study Fields #######
# NCTId,BriefTitle,Condition,Keyword,EligibilityCriteria 	

####### New Fields to add #######
# LocationStatus, Phase

####### Range Min_MAX ######
# 1 to 1000

####### Format #######
# CSV

url = 'https://clinicaltrials.gov/api/query/study_fields?expr=&fields=NCTId%2CBriefTitle%2CCondition%2CKeyword%2CEligibilityCriteria+%09&min_rnk=1&max_rnk=1000&fmt=csv'
session = requests.Session()
retry   = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://' , adapter)
session.mount('https://', adapter)

clinicaltrials = session.get(url)
print('Download Request Status: ', clinicaltrials.status_code)

csv_file = open('/content/'+str(1)+'-batch.csv', 'wb')
csv_file.write(clinicaltrials.content)
csv_file.close()

df = pd.read_csv(r'/content/1-batch.csv', skiprows=10)
print(df.shape)
df.head()

Download Request Status:  200
(1000, 6)


,Rank,NCTId,BriefTitle,Condition,Keyword,EligibilityCriteria
0,1,NCT05235100,Preoperative IMRT With Concurrent Apatinib for...,"Sarcoma,Soft Tissue|Extremity|Trunk|Intensity-...",NaN,Inclusion Criteria:||Age older than 18-yo.|His...
1,2,NCT05235087,Bovine Atelocollagen Skin Sensitization Test,Healthy,NaN,"Inclusion Criteria:||Healthy volunteers, of ei..."
2,3,NCT05235074,OH2 Oncolytic Viral Therapy in Central Nervous...,Central Nervous System Tumors,Oncolytic Virus,"Inclusion Criteria:||Age ≥ 18 years old, male ..."
3,4,NCT05235061,HOme-Based Self-management and COgnitive Train...,"Epilepsy, Traumatic|Brain Injuries, Traumatic","Epilepsy|Epilepsy, Post-Traumatic|Traumatic Br...",Inclusion Criteria/PTE Participants:||Age 18+|...
4,5,NCT05235048,Single Tooth Extraction in Damaged Alveoli and...,Tooth Extraction Status Nos,Alveolar ridge preservation|Tooth extraction|D...,Inclusion Criteria:||Systemically healthy. No ...


In [67]:
# Convert the EligibilityCriteria to string then lowercase
EligibilityCriteria = df['EligibilityCriteria'].astype(str).str.lower().tolist()
EligibilityCriteria[:10]

["inclusion criteria:||age older than 18-yo.|histology proven soft tissue sarcoma of truncal or extremity, deemed appropriate for preoperative radiotherapy and conservative surgery by multidisciplinary discussion.|ecog 0-3|histology reviewed by reference pathologist|lesion can be assessed|can tolerate radiotherapy and apatinib|agree contraception.|informed consent: all patients must sign a document of informed consent indicating their understanding of the investigational nature and risks of the study before any protocol related studies are performed||exclusion criteria:||no gross tumor post-resection in other center.|contraindications to apatinib, including allergic to apatinib, active bleeding, ulcer, enteric perforation, enteric obstruction, uncontrolled hypertension, grade 3 to 4 cardiac insufficiency (per nyha criteria), and severe hepatic or renal insufficiency (grade 4), etc.|dermatofibrosarcoma protuberans(dfsp), desmoids, etc.|benign histology|secondary cancer within 5 years (e

In [68]:
# Seperate the InclusionCriteria from the ExclusionCriteria
InclusionCriteria = [txt[21:txt.find('exclusion criteria')-2] for txt in EligibilityCriteria]
InclusionCriteria[:10]

['age older than 18-yo.|histology proven soft tissue sarcoma of truncal or extremity, deemed appropriate for preoperative radiotherapy and conservative surgery by multidisciplinary discussion.|ecog 0-3|histology reviewed by reference pathologist|lesion can be assessed|can tolerate radiotherapy and apatinib|agree contraception.|informed consent: all patients must sign a document of informed consent indicating their understanding of the investigational nature and risks of the study before any protocol related studies are performed',
 'healthy volunteers, of either sex, aged at least 18 years.|volunteers must be capable of understanding and following directions in english.|subjects who are healthy and have normal skin on the volar surface of the forearm.|subjects willing and able to comply with the requirements of the study.|subjects willing and able to comply with the follow-up requirements.|subjects willing and able to give written and verbal informed consent.',
 'age ≥ 18 years old, ma

In [70]:
# Seperate the ExclusionCriteria from the InclusionCriteria
ExclusionCriteria = [txt[txt.find('exclusion criteria')+21:] for txt in EligibilityCriteria]
ExclusionCriteria[:10]

["no gross tumor post-resection in other center.|contraindications to apatinib, including allergic to apatinib, active bleeding, ulcer, enteric perforation, enteric obstruction, uncontrolled hypertension, grade 3 to 4 cardiac insufficiency (per nyha criteria), and severe hepatic or renal insufficiency (grade 4), etc.|dermatofibrosarcoma protuberans(dfsp), desmoids, etc.|benign histology|secondary cancer within 5 years (except cervical carcinoma in situ or early-stage skin basal cell carcinoma)|sts can be cured by extensive operation alone.|previous irradiation to the same area|radiological evidence of distant metastases|other contraindications, can't tolerate operation or other treatment needed in this study.|neoadjuvant chemotherapy given or planned.",
 "subjects who are pregnant, nursing or intend to become pregnant.|current treatment by a physician for allergy, unless physician consulted by investigator and participation was approved.|participation in a repeat insult patch test (rip

In [71]:
# Split each Trial InclusionCriteria into list of lists of the InclusionCriteria
InclusionCriteriaList = list(map(lambda txt : txt.split("|"), InclusionCriteria))
InclusionCriteriaList = [list(filter(lambda txt: txt!= '', ltexts)) for ltexts in InclusionCriteriaList] # Delete the empty string generated by successive || (sometimes they seperate with | or ||)
InclusionCriteriaList[:2]

[['age older than 18-yo.',
  'histology proven soft tissue sarcoma of truncal or extremity, deemed appropriate for preoperative radiotherapy and conservative surgery by multidisciplinary discussion.',
  'ecog 0-3',
  'histology reviewed by reference pathologist',
  'lesion can be assessed',
  'can tolerate radiotherapy and apatinib',
  'agree contraception.',
  'informed consent: all patients must sign a document of informed consent indicating their understanding of the investigational nature and risks of the study before any protocol related studies are performed'],
 ['healthy volunteers, of either sex, aged at least 18 years.',
  'volunteers must be capable of understanding and following directions in english.',
  'subjects who are healthy and have normal skin on the volar surface of the forearm.',
  'subjects willing and able to comply with the requirements of the study.',
  'subjects willing and able to comply with the follow-up requirements.',
  'subjects willing and able to give 

In [73]:
# Split each Trial ExclusionCriteria into list of lists of the ExclusionCriteria
ExclusionCriteriaList = list(map(lambda txt : txt.split("|"), ExclusionCriteria))
ExclusionCriteriaList = [list(filter(lambda txt: txt!= '', ltexts)) for ltexts in ExclusionCriteriaList] # Delete the empty string generated by successive || (sometimes they seperate with | or ||)
ExclusionCriteriaList[:2]

[['no gross tumor post-resection in other center.',
  'contraindications to apatinib, including allergic to apatinib, active bleeding, ulcer, enteric perforation, enteric obstruction, uncontrolled hypertension, grade 3 to 4 cardiac insufficiency (per nyha criteria), and severe hepatic or renal insufficiency (grade 4), etc.',
  'dermatofibrosarcoma protuberans(dfsp), desmoids, etc.',
  'benign histology',
  'secondary cancer within 5 years (except cervical carcinoma in situ or early-stage skin basal cell carcinoma)',
  'sts can be cured by extensive operation alone.',
  'previous irradiation to the same area',
  'radiological evidence of distant metastases',
  "other contraindications, can't tolerate operation or other treatment needed in this study.",
  'neoadjuvant chemotherapy given or planned.'],
 ['subjects who are pregnant, nursing or intend to become pregnant.',
  'current treatment by a physician for allergy, unless physician consulted by investigator and participation was appro

In [74]:
df['InclusionCriteriaList'] = InclusionCriteriaList
df['ExclusionCriteriaList'] = ExclusionCriteriaList
df.head()

,Rank,NCTId,BriefTitle,Condition,Keyword,EligibilityCriteria,InclusionCriteriaList,ExclusionCriteriaList
0,1,NCT05235100,Preoperative IMRT With Concurrent Apatinib for...,"Sarcoma,Soft Tissue|Extremity|Trunk|Intensity-...",NaN,Inclusion Criteria:||Age older than 18-yo.|His...,"[age older than 18-yo., histology proven soft ...",[no gross tumor post-resection in other center...
1,2,NCT05235087,Bovine Atelocollagen Skin Sensitization Test,Healthy,NaN,"Inclusion Criteria:||Healthy volunteers, of ei...","[healthy volunteers, of either sex, aged at le...","[subjects who are pregnant, nursing or intend ..."
2,3,NCT05235074,OH2 Oncolytic Viral Therapy in Central Nervous...,Central Nervous System Tumors,Oncolytic Virus,"Inclusion Criteria:||Age ≥ 18 years old, male ...","[age ≥ 18 years old, male or female;, phase i ...","[1.28 days before enrollment, subjects partici..."
3,4,NCT05235061,HOme-Based Self-management and COgnitive Train...,"Epilepsy, Traumatic|Brain Injuries, Traumatic","Epilepsy|Epilepsy, Post-Traumatic|Traumatic Br...",Inclusion Criteria/PTE Participants:||Age 18+|...,"[e participants:, age 18+, diagnosis of post t...","[e participants:, subjects self-reporting a de..."
4,5,NCT05235048,Single Tooth Extraction in Damaged Alveoli and...,Tooth Extraction Status Nos,Alveolar ridge preservation|Tooth extraction|D...,Inclusion Criteria:||Systemically healthy. No ...,[systemically healthy. no contraindications to...,[presence of frank purulence or acute abscess ...


In [76]:
df.to_excel('df.xlsx')
files.download('df.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>